<a href="https://colab.research.google.com/github/amirmohammadkalateh/Fetal_Health/blob/main/fetal_health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [48]:
df = pd.read_csv('fetal_health.csv')
X = df.drop('fetal_health', axis=1)
y = df['fetal_health']
y = to_categorical(y - 1)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [50]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [51]:
inputs = Input(shape=(X_train.shape[1],))


x = BatchNormalization()(inputs)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)

x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
x = BatchNormalization()(x)

x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)

outputs = Dense(3, activation='softmax')(x)


model = Model(inputs=inputs, outputs=outputs)


In [52]:
optimizer = Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [53]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


y_classes = np.argmax(y_train, axis=1)
class_weights = compute_class_weight(
    'balanced', classes=np.unique(y_classes), y=y_classes
)
class_weights = dict(enumerate(class_weights))

In [54]:
early_stop = EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    class_weight=class_weights,
    callbacks=[early_stop]
)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.3850 - loss: 1.5571 - val_accuracy: 0.7765 - val_loss: 0.8466
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5076 - loss: 1.0771 - val_accuracy: 0.8294 - val_loss: 0.6752
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6312 - loss: 0.8228 - val_accuracy: 0.8412 - val_loss: 0.5347
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6764 - loss: 0.7589 - val_accuracy: 0.8559 - val_loss: 0.4662
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7041 - loss: 0.7102 - val_accuracy: 0.8500 - val_loss: 0.4189
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7218 - loss: 0.6669 - val_accuracy: 0.8471 - val_loss: 0.3985
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7457 - loss: 0.6728 - val_accuracy: 0.8500 - val_loss: 0.4053
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7585 - loss: 0.6293 - val_accuracy: 0.8471 - 

In [55]:

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8696 - loss: 0.3827 
Test Accuracy: 87.56%
